<a href="https://colab.research.google.com/github/ramkumar27072006/AMR_2nd/blob/main/CNN_BiLSTM_Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving protein2_dataset.csv to protein2_dataset.csv


In [2]:
import pandas as pd

# Assuming the file name is 'gene sequence(AMR dataset).csv', use the exact file name
df = pd.read_csv('protein2_dataset.csv')

# Show the first few rows of the dataframe
print(df.head())

  Protein_ID                                           Sequence  Label
0     A5U8S6  MLRTMLKSKIHRATVTCADLHYVGSVTIDADLMDAADLLEGEQVTI...      1
1     D0NPN8  MRLAIMLSATAVAINFATSSAIDQTKVLVYGTPAHYIHDSAGRRLL...      1
2     D0P3S7  MRSLLLTVLLNLVVLLATTGAVSSNLNTAVNYASTSKIRFLSTEYN...      1
3     E2DWQ7  MRLAIMLSATAVAINFATCSAIDQTKVLVYGTPAHYIHDSAGRRLL...      1
4     G4V4G2  MDIDNKPRIHWACRRGMRELDISIMPFFEHDYDTLSDDDKRNFIRL...      1


In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Bidirectional, Dense, Flatten, Dropout, MaxPooling1D

# Define amino acids
AA = "ARNDCQEGHILKMFPSTWYV"  # 20 standard amino acids

def one_hot_encode(sequence, max_len=200):
    """One-hot encodes a protein sequence"""
    enc = np.zeros((max_len, len(AA)))  # Create empty matrix

    for i, aa in enumerate(sequence[:max_len]):  # Limit to max_len
        if aa in AA:
            enc[i, AA.index(aa)] = 1  # One-hot encode

    return enc

In [15]:
# Convert all sequences
X_onehot = np.array([one_hot_encode(seq) for seq in df["Sequence"]])
y = df["Label"].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_onehot, y, test_size=0.2, random_state=42)

In [16]:
# Define CNN-BiLSTM model with modifications
model = Sequential([
    Conv1D(filters=128, kernel_size=5, activation="relu", input_shape=(200, 20)), # Increased filters, kernel size
    MaxPooling1D(pool_size=2), # Added MaxPooling for downsampling
    Bidirectional(LSTM(128, return_sequences=True)), # Increased LSTM units
    Bidirectional(LSTM(64)),
    Dense(128, activation="relu"), # Increased Dense units
    Dropout(0.5), # Increased dropout rate
    Dense(1, activation="sigmoid")  # Binary classification
])

# Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train model with more epochs and early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5) # Early stopping to prevent overfitting
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 36s 634ms/step - accuracy: 0.8039 - loss: 0.5350 - val_accuracy: 0.8353 - val_loss: 0.4473
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 27s 631ms/step - accuracy: 0.8619 - loss: 0.3998 - val_accuracy: 0.8382 - val_loss: 0.4223
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 28s 644ms/step - accuracy: 0.8833 - loss: 0.3477 - val_accuracy: 0.8441 - val_loss: 0.3756
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 41s 647ms/step - accuracy: 0.8969 - loss: 0.3063 - val_accuracy: 0.8588 - val_loss: 0.3469
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 27s 632ms/step - accuracy: 0.9084 - loss: 0.2882 - val_accuracy: 0.8618 - val_loss: 0.3382
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 40s 608ms/step - accuracy: 0.9038 - loss: 0.2838 - val_accuracy: 0.8618 - val_loss: 0.3365
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 27s 633ms/step - accuracy: 0.8973 - loss: 0.2918 - val_accuracy: 0.8618 - val_loss: 0.3364
Epoch 8/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 27s 638ms/step - accuracy: 0.8994 - loss: 0.2900 - val_accu

In [51]:
# Test on a new sequence
new_seq = "MGGRWSKSSIVGWPAIRERIRRTDPAADGVGAVSRDLEKHGAITSSNTRGTNADCAWLEAQEESEEVGFPVRPQVPLRPMTYKGALDLSHFLKEKGG"
new_seq_encoded = one_hot_encode(new_seq).reshape(1, 200, 20)

# Predict
prob = model.predict(new_seq_encoded)[0][0]
pred_class = 1 if prob > 0.5 else 0

print(f"Predicted Class: {'Virulent (1)' if pred_class == 1 else 'Non-Virulent (0)'}")
print(f"Prediction Confidence: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted Class: Virulent (1)
Prediction Confidence: 0.8128
